In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [3]:
import pandas as pd
import os
import re
import codecs
from IPython.display import display, Math
from six.moves import cPickle as pickle
import string
from PIL import Image
import numpy as np
import h5py
# Config the matplotlib backend as plotting inline in IPython
%matplotlib inline

In [4]:
width = None
pd.options.display.max_rows = 600
pd.options.display.max_columns = width
pd.options.display.max_colwidth = 600
pd.options.display.width = width
pd.options.display.max_seq_items = None
pd.options.display.expand_frame_repr = False
pd.options.display.colheader_justify = 'left'

In [5]:
import data_commons as dtc
import dl_commons as dlc
import viz_commons as vc
from viz_commons import VisualizeDir, DiffParams, VisualizeStep

Using TensorFlow backend.


In [19]:
vd = VisualizeDir(os.path.expanduser('/zpool_3TB/i2l/tb_metrics/2017-12-25 21-04-15 PST 140K_noRegroup_score89.09/test_runs/step_00167526/store_2'))

Loaded /zpool_3TB/i2l/tb_metrics/2017-12-25 21-04-15 PST 140K_noRegroup_score89.09/test_runs/step_00167526/store_2/hyper.pkl and /zpool_3TB/i2l/tb_metrics/2017-12-25 21-04-15 PST 140K_noRegroup_score89.09/test_runs/step_00167526/store_2/args.pkl
Loaded ../data/dataset5/training_56/df_train.pkl (114408, 3)
Loaded ../data/dataset5/training_56/df_test.pkl (14280, 3)
Loaded ../data/dataset5/training_56/df_valid.pkl (14280, 3)


In [8]:
vd.view_snapshots()

Num Snapshots: 1
Snapshots = [(167526, 146585.25)]


In [9]:
vd.view_steps()

num epoch_steps = 2
epoch_steps = [(167526, 146585), (167526, 146585)]
all_steps = [(167526, 146585), (167526, 146585)]


In [25]:
vs = VisualizeStep(vd, 'test', 167526)
df_strs = vs.strs('predicted_ids', trim=True, wrap_strs=True)

In [33]:
num_hits = df_strs[df_strs.edit_distance == 0].count()[0]
# df_strs.count()
df_hits = df_strs.iloc[0:num_hits-1].sort_values(by=['len'], ascending=False)

In [34]:
df_hits

,_id,edit_distance,len,predicted_ids
4019,1230,0.0,147,$ds^{2}=-\frac {t^{2}}{(t^{2}+r_{-}^{2})(t^{2}-r_{+}^{2})}dt^{2}+t^{2}(d\phi +\frac {r_{+}r_{-}}{t^{2}}dr)^{2}+\frac {(t^{2}+r_{-}^{2})(t^{2}-r_{+}^{2})}{t^{2}}dr^{2}.$
14016,3561,0.0,147,$\Delta ^{\alpha \beta }=\frac {1}{k^{2}+\Pi _{t}}\left( \delta ^{\alpha \beta }-\frac {p_{0}^{\alpha }k^{\beta }+k^{\alpha }p_{0}^{\beta }}{p_{0}\cdot k}-\frac {k^{2}}{k^{2}-\Pi _{\ell }}(\Pi _{t}+\Pi _{\ell })\frac {p_{0}^{\alpha }p_{0}^{\beta }}{(p_{0}\cdot k)^{2}}\right) \; .$
3993,1239,0.0,147,"${\cal L}_{Y}=-\sum _{i,j=1}^{G}\left[ \hat {y}_{ij}^{e}\bar {L}_{L}^{i}\phi \, E_{R}^{j}+\hat {y}_{ij}^{d}\bar {Q}_{L}^{i}\phi \, D_{R}^{j}+\hat {y}_{ij}^{u}\bar {Q}_{L}^{i}\tilde {\phi }\, U_{R}^{j}+\mathrm {h.c.}\right] ,$"
8844,1132,0.0,143,"$ds^{2}~=~\alpha ^{\prime }\left( \frac {u^{2}h(u)}{R^{2}}~e^{\gamma A}dx_{0}^{2}+\frac {u^{2}}{R^{2}}~e^{\gamma C}dx_{i}^{2}+\frac {R^{2}}{u^{2}h(u)}~e^{\gamma B}du^{2}+R^{2}~e^{\gamma D}d\Omega _{5}^{2}\right) ~,$"
9585,2987,0.0,143,"$\langle J_{\mu _{1}}^{a_{1}}(P_{1})\ldots J_{\mu _{n}}^{a_{n}}(P_{n})\rangle _{T}\approx (-i)^{n}\frac {NT^{2}}{12}\delta \Gamma _{\mu _{1}\ldots \mu _{n}}^{a_{1}\ldots a_{n}}(P_{1},\ldots ,P_{n})+O(\frac {1}{f_{\pi }^{2}})\,$"
539,3620,0.0,140,"$g_{q}(y,M)={\frac {M}{18\pi }}x_{a}x_{b}\[f_{q}^{(a)}(x_{a},M^{2})f_{\bar {q}}^{(b)}(x_{b},M^{2})+f_{\bar {q}}^{(a)}(x_{a},M^{2})f_{q}^{(b)}(x_{b},M^{2})]\,$"
6863,1601,0.0,139,"$D_{\mu \nu }^{ab}(p,p_{3})=\frac {\delta ^{ab}\delta ^{a3}}{p^{2}-p_{3}^{2}+i\epsilon }\left[ -g_{\mu \nu }+p_{\mu }p_{\nu }\left( (1-\delta _{p_{3},0})\frac {1}{p_{3}^{2}}+\delta _{p_{3},0}(1-\xi )\frac {1}{p^{2}+i\epsilon }\right) \right] $"
5380,2522,0.0,138,"$\frac {1}{\operatorname {sin}\theta }\frac {\partial }{\partial \theta }\left( \operatorname {sin}\theta \frac {\partial Z}{\partial \theta }\right) +\frac {1}{\operatorname {sin}^{2}\theta }\frac {\partial ^{2}Z}{\partial \varphi ^{2}}-\left( \frac {c_{1}}{\operatorname {cos}^{2}\frac {\theta }{2}}+\frac {c_{2}}{\operatorname {sin}^{2}\frac {\theta }{2}}\right) Z=-AZ,$"
4883,672,0.0,138,$p_{1}^{2}p_{2}^{2}-(p_{1}\cdot p_{2})^{2}=-\frac {1}{4}(p_{1}^{4}+p_{2}^{4}+p_{5}^{4}-2p_{1}^{2}p_{2}^{2}-2p_{1}^{2}p_{5}^{2}-2p_{2}^{2}p_{5}^{2})=0$
523,3596,0.0,136,"$\langle p_{A}^{\prime }k_{1}\rangle \langle k_{2}p_{B}\rangle \langle p_{B}^{\prime }p_{A}\rangle =\langle p_{A}k_{1}\rangle \langle k_{2}p_{B}^{\prime }\rangle \langle p_{B}p_{A}^{\prime }\rangle =\langle p_{A}^{\prime }k_{1}\rangle \langle k_{2}p_{B}^{\prime }\rangle \langle p_{B}p_{A}\rangle \, ,$"


In [ ]:
nd_alpha = vs.alpha(1, invert_alpha=False, gamma_correction=1, cmap='gist_gray')  # gist_gray, gist_yarg, gist_heat
# nd_alpha = vs.alpha(17, invert_alpha=False, gamma_correction=1, cmap='gist_yarg')

CustomConvnetImageProcessor: image_batch_shape = (1, 128, 1088, 1), image_shape_unframed = [128, 1088, 1]


<IPython.core.display.Math object>

ds_{4}^{2}=-f(R)\left( 1-\frac {2m}{R}\right) dt^{2}+f^{-1}(R)\left[ \left( 1-\frac {2m}{R}\right) ^{-1}dR^{2}+R^{2}d\Omega _{2}^{2}\right] 


<IPython.core.display.Math object>

ds_{4}^{2}=-f(R)\left( 1-\frac {2m}{R}\right) dt^{2}+f^{-1}(R)\left[ \left( 1-\frac {2m}{R}\right) ^{-1}dR^{2}+R^{2}d\Omega _{2}^{2}\right] 
